In [4]:
import datetime
import sys
import os

import requests
import json

import jaydebeapi

from piccolo.table import Table
from piccolo.columns import Varchar, Integer, Timestamp



In [28]:

sys.path.append('../../jupyter_cell_magics')

import jflask
import jsql_dev as jsql
import jedit

jar_folder = os.environ['SQL_JARS']
jar_files = [os.path.join(jar_folder, jar_file) for jar_file in os.listdir(jar_folder)]
my_conn = jsql.Connection('my_conn', jaydebeapi.connect, *["org.postgresql.Driver",
                          "jdbc:postgresql://localhost:5432/my_app",
                          ["postgres", "pwdPostgreSQL"], 
                            jar_files
                          ,], verbose = False, keep_closed = False)

my_conn.open()
my_conn.connection.jconn.setAutoCommit(False)

[0.002s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016cf1c000-0x000000016cf28000).
[0.002s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [61]:
import webbrowser
url = 'http://localhost:5000/ping'
webbrowser.open_new_tab(url)


True

going to ping...


127.0.0.1 - - [13/Oct/2021 19:49:39] "GET /ping HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2021 19:49:39] "GET /favicon.ico HTTP/1.1" 404 -


In [3]:
r = requests.post('http://localhost:5000/ping')

In [6]:
r.json()

{'some': 'data'}

In [1]:
%%writefile piccolo_conf.py
# piccolo_conf.py
from piccolo.engine.postgres import PostgresEngine


DB = PostgresEngine(config={
    'host': 'localhost',
    'database': 'my_app',
    'user': 'postgres',
    'password': 'pwdPostgreSQL'
})

Overwriting piccolo_conf.py


In [15]:
class time_log(Table):
    jira_id = Varchar(length=20)
    description = Varchar(length=400) 
    log_time = Timestamp()
    
#time_log.create_table().run_sync()


In [16]:
time_log.select().run_sync()

[{'id': 1,
  'jira_id': 'test_ticket',
  'description': 'test',
  'log_time': datetime.datetime(2021, 10, 19, 19, 27, 15, 70704)},
 {'id': 2,
  'jira_id': 'test_ticket',
  'description': 'test',
  'log_time': datetime.datetime(2021, 10, 19, 19, 27, 19, 574387)},
 {'id': 3,
  'jira_id': 'ABC-2',
  'description': '',
  'log_time': datetime.datetime(2021, 10, 19, 19, 27, 31, 302530)},
 {'id': 4,
  'jira_id': 'ABC-2',
  'description': '',
  'log_time': datetime.datetime(2021, 10, 19, 19, 27, 59, 612246)},
 {'id': 5,
  'jira_id': 'ABC-2',
  'description': '',
  'log_time': datetime.datetime(2021, 10, 19, 19, 28, 54, 432035)},
 {'id': 6,
  'jira_id': 'ABC-1',
  'description': '',
  'log_time': datetime.datetime(2021, 10, 19, 19, 34, 55, 856497)},
 {'id': 7,
  'jira_id': 'ABC-1',
  'description': '',
  'log_time': datetime.datetime(2021, 10, 19, 19, 35, 19, 281902)},
 {'id': 8,
  'jira_id': 'ABC-2',
  'description': '',
  'log_time': datetime.datetime(2021, 10, 19, 19, 35, 22, 613543)},
 {'id

In [17]:
# starting docker
#!docker start myPostgreSQL
#!docker ps -a

In [18]:
class task(Table):
    jira_id = Varchar(length=20)
    name = Varchar(length=50)
    description = Varchar(length=200)
    project = Varchar(length=40)
    last_used = Timestamp(null=True)
    closed = Timestamp(null=True)

#task.create_table().run_sync()

In [7]:
task_import_example = """
ABC-1;task 1;;project A
ABC-2;task 2;;project A
ABC-3;task 3;;project B
"""

In [8]:
task_keys = ['jira_id', 'name', 'description', 'project']

In [13]:
task_data_rows = [r.split(';') for r in task_import_example.split('\n') if ';'in r]
task_data_rows

[['ABC-1', 'task 1', '', 'project A'],
 ['ABC-2', 'task 2', '', 'project A'],
 ['ABC-3', 'task 3', '', 'project B']]

In [37]:
for task_data in task_data_rows:
    task_dict = dict(zip(task_keys, task_data))
    task.insert(task(**task_dict, last_used=None, closed=None)).run_sync()

In [38]:
task.select().run_sync()

[{'id': 1,
  'jira_id': 'ABC-1',
  'name': 'task 1',
  'description': '',
  'project': 'project A',
  'last_used': None,
  'closed': None},
 {'id': 2,
  'jira_id': 'ABC-2',
  'name': 'task 2',
  'description': '',
  'project': 'project A',
  'last_used': None,
  'closed': None},
 {'id': 3,
  'jira_id': 'ABC-3',
  'name': 'task 3',
  'description': '',
  'project': 'project B',
  'last_used': None,
  'closed': None}]

In [35]:
%%alter conn --commit
drop table task;

COMMIT


[0]

In [75]:
%%alter conn --commit
drop table time_log;

COMMIT


[0]

In [19]:
def get_tasks(closed=False):
    ret = {}
    ret['tasks'] = task.select().run_sync()
    return json.dumps(ret)
get_tasks()

'{"tasks": [{"id": 1, "jira_id": "ABC-1", "name": "task 1", "description": "", "project": "project A", "last_used": null, "closed": null}, {"id": 2, "jira_id": "ABC-2", "name": "task 2", "description": "", "project": "project A", "last_used": null, "closed": null}, {"id": 3, "jira_id": "ABC-3", "name": "task 3", "description": "", "project": "project B", "last_used": null, "closed": null}]}'

In [22]:
my_app = jflask.FlaskApp("appi")
my_app.run()

Stopping app
Starting app
 * Serving Flask app 'appi' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [23]:
#my_app.stop()
my_app.ping()

Pinging


127.0.0.1 - - [25/Oct/2021 18:29:37] "GET / HTTP/1.1" 200 -


b'appi running'

In [34]:
def get_tasks():
    ret = {}
    time_logs = time_log.select().run_sync()
    ret['tasks'] = [{'jira_id' : jira_id} for jira_id in set([l['jira_id'] for l in time_logs])]
    return json.dumps(ret)
get_tasks()
    

my_app.addMethodFlush('/get_tasks', get_tasks)

r = requests.get('http://localhost:5000/get_tasks')
r.json()

127.0.0.1 - - [25/Oct/2021 18:37:02] "GET /get_tasks HTTP/1.1" 200 -


{'tasks': [{'jira_id': 'test_ticket'},
  {'jira_id': 'UNIT-A'},
  {'jira_id': 'UUSI-2'},
  {'jira_id': 'ssss'},
  {'jira_id': 'ABC-3'},
  {'jira_id': 'NEW'},
  {'jira_id': 'ABC-1'},
  {'jira_id': 'sss'},
  {'jira_id': 'ss'},
  {'jira_id': 'ABC-2'},
  {'jira_id': 'UUSI-1'}]}

In [36]:
insert_log_test_data = json.loads(insert_log_test_json)
def insert_log(data_json):
    data = json.loads(data_json)
    jira_id = data['jira_id']
    description = data['description']
    ret = time_log.insert(time_log(description=description, jira_id=jira_id, log_time=datetime.datetime.now())).run_sync()
    return json.dumps(ret)

my_app.addPostMethod("/insert_log", insert_log)

In [ ]:
insert_log_test_json = """
{"id":2,"jira_id":"ABC-2","name":"task 2","description":"","project":"project A","last_used":null,"closed":null}
"""
insert_log(insert_log_test_json)

In [37]:
r = requests.post('http://localhost:5000/insert_log', data = insert_log_test_json)

response_json = [{"id": 73}]
response = <Response 17 bytes [200 OK]>


127.0.0.1 - - [25/Oct/2021 18:37:55] "POST /insert_log HTTP/1.1" 200 -


In [ ]:
def get_logs():
    ret = {}
    ret['logs'] = time_log.select().order_by(time_log.log_time, ascending=False).run_sync()
    return json.dumps(ret, default=str)

logs = get_logs()

my_app.addMethodFlush('/get_logs', get_logs)

r = requests.get('http://localhost:5000/get_logs')

r.json()

In [40]:
%%jedit -h -w test_insert_log.html
<button onclick="test_insert()">insert_log</button>
<table id="logs"></table>

<script src="insert_log.js"></script>

<script>
function test_insert() {
    test_data = JSON.parse('{"jira_id":"UNIT-A", "description":"unit test"}')
    insert_log(test_data)
    refresh_logs()
}
</script>

127.0.0.1 - - [25/Oct/2021 18:40:55] "OPTIONS /insert_log HTTP/1.1" 200 -


response_json = [{"id": 74}]
response = <Response 17 bytes [200 OK]>


127.0.0.1 - - [25/Oct/2021 18:40:55] "POST /insert_log HTTP/1.1" 200 -


In [41]:
%%jedit -w insert_log.js -t test_insert_log.html -o temp.html
function insert_log(data) {
var insert_log_url = "http://localhost:5000/insert_log";
var xhr = new XMLHttpRequest();
xhr.open("POST", insert_log_url, false);
xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");
var data_json = JSON.stringify(data);
xhr.send(data_json);
console.log(xhr.responseText);

}

Opening page file:////Users/rook/Documents/github/python/hour_reporting/temp.html in the browser


127.0.0.1 - - [25/Oct/2021 18:41:08] "OPTIONS /insert_log HTTP/1.1" 200 -


response_json = [{"id": 75}]
response = <Response 17 bytes [200 OK]>


127.0.0.1 - - [25/Oct/2021 18:41:08] "POST /insert_log HTTP/1.1" 200 -


In [42]:
%%jedit -h -w test_tasks.html
<button onclick="test_refresh_tasks()">test refresh tasks</button>
<div id="tasks" class="tasks"></div>
<div id="debug"></div>

<style>
.tasks  {background-color: #F2E4D6;}
.task  {background-color: #F2E4F6;}
.task   {color: blue; float: left;}
.task:hover {
    cursor: pointer;
    background-color: #D2E4F6;
}
.task {
  margin: 5px 5px 5px 5px;
}
</style>

<script src="tasks.js"></script>

<script>
function test_refresh_tasks() {
    refresh_tasks()
}
refresh_tasks()
</script>

127.0.0.1 - - [25/Oct/2021 18:41:21] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:21] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:24] "GET /get_tasks HTTP/1.1" 200 -


In [45]:
%%jedit -w logs.js -t test_main.html -o temp.html

function element_from_log(log) {
//    console.log(log)
    var log_el = document.createElement("tr");
    log_el.className = 'log';
    var jira_id = document.createElement("td");
    jira_id.innerHTML = log['jira_id'];
    log_el.appendChild(jira_id);
    var description = document.createElement("td");
    description.innerHTML = log['description'];
    log_el.append(description)
    var log_time = document.createElement("td");
    log_time.innerHTML = log['log_time'];
    log_el.append(log_time)
    document.getElementById("logs").appendChild(log_el)
}

function refresh_logs() {
var url = "http://localhost:5000/get_logs";

var xhr = new XMLHttpRequest();
xhr.open("GET", url, true);

xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");

xhr.onreadystatechange = function () {
   if (xhr.readyState === 4) {
      console.log(xhr.status);
       logs_response = JSON.parse(xhr.responseText);
        logs = logs_response['logs'];
       document.getElementById("logs").innerHTML = "";
        logs.forEach(element_from_log);

   }};

xhr.send(null);
}



Opening page file:////Users/rook/Documents/github/python/hour_reporting/temp.html in the browser


127.0.0.1 - - [25/Oct/2021 18:42:42] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:42] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:42] "OPTIONS /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:42] "GET /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:46] "OPTIONS /insert_log HTTP/1.1" 200 -


response_json = [{"id": 77}]
response = <Response 17 bytes [200 OK]>


127.0.0.1 - - [25/Oct/2021 18:42:46] "POST /insert_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:46] "OPTIONS /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:46] "GET /get_logs HTTP/1.1" 200 -


In [44]:
%%jedit -h  -w test_logs.html
<p>Hour logs</p>

<button onclick="refresh_logs()">test refresh logs</button>

<table id="logs"></table>

<script src="logs.js"></script>
<script>
function test_refresh_logs() {
    refresh_logs()
}
refresh_logs()
</script>

127.0.0.1 - - [25/Oct/2021 18:42:15] "OPTIONS /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:15] "GET /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:42:17] "GET /get_logs HTTP/1.1" 200 -


In [43]:
%%jedit -w tasks.js -t test_main.html -o temp.html

function element_from_task(task) {
//    console.log(task)
    var task_el = document.createElement("button");
    task_el.jira_id = task.jira_id
    task_el.className = 'task';
    var jira_id = document.createElement("p");
    jira_id.innerHTML = task['jira_id'];
    task_el.appendChild(jira_id);
    function task_el_onclick() {
//        alert (this.jira_id)
        if (this.jira_id == 'NEW') {
            this.jira_id = prompt("New jira_id");
        }
        this.description = prompt("description");
        insert_log(this);
        refresh_logs();
    }
    task_el.onclick = task_el_onclick; 
    document.getElementById("tasks").appendChild(task_el)
}

function refresh_tasks() {
var url = "http://localhost:5000/get_tasks";
var xhr = new XMLHttpRequest();
xhr.open("GET", url, false);
xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");
xhr.send(null);
tasks_response = JSON.parse(xhr.responseText);
tasks = tasks_response['tasks'];
document.getElementById("tasks").innerHTML = "";
tasks.forEach(element_from_task);
element_from_task(JSON.parse('{ "jira_id":"NEW", "description":"nil" }'))
};


Opening page file:////Users/rook/Documents/github/python/hour_reporting/temp.html in the browser


127.0.0.1 - - [25/Oct/2021 18:41:40] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:40] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:40] "OPTIONS /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:40] "GET /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:46] "OPTIONS /insert_log HTTP/1.1" 200 -


response_json = [{"id": 76}]
response = <Response 17 bytes [200 OK]>


127.0.0.1 - - [25/Oct/2021 18:41:46] "POST /insert_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:46] "OPTIONS /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:41:46] "GET /get_logs HTTP/1.1" 200 -


In [50]:
%%jedit -h -o hour_tasks_demo.html -w test_main.html
<h1> Pääohjelma </h1>

<div id="tasks"></div>
<table id="logs"></table>

<script src="insert_log.js"></script>
<script src="tasks.js"></script>
<script src="logs.js"></script>


<script>
refresh_tasks()
refresh_logs()
</script>

<h1> Pääohjelma </h1>

<div id="tasks"></div>
<table id="logs"></table>

<script src="insert_log.js"></script>
<script src="tasks.js"></script>
<script src="logs.js"></script>


<script>
refresh_tasks()
refresh_logs()
</script>
Opening page file:////Users/rook/Documents/github/python/hour_reporting/hour_tasks_demo.html in the browser


127.0.0.1 - - [25/Oct/2021 18:48:47] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:48:47] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:48:47] "OPTIONS /get_logs HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:48:47] "GET /get_logs HTTP/1.1" 200 -


In [51]:
with open('test_main.html', 'r') as f:
    hour_interface_html = f.read()
hour_interface_html

'<h1> Pääohjelma </h1>\n\n<div id="tasks"></div>\n<table id="logs"></table>\n\n<script src="insert_log.js"></script>\n<script src="tasks.js"></script>\n<script src="logs.js"></script>\n\n\n<script>\nrefresh_tasks()\nrefresh_logs()\n</script>\n'

In [53]:
import webbrowser

In [54]:
my_app.addPage("/hour_interface", hour_interface_html)
webbrowser.open_new_tab('http://localhost:5000/hour_interface')


True

127.0.0.1 - - [25/Oct/2021 18:49:22] "GET /hour_interface HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 18:49:22] "GET /insert_log.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2021 18:49:22] "GET /tasks.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2021 18:49:22] "GET /logs.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2021 18:49:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2021 18:49:35] "GET /insert_log.js HTTP/1.1" 404 -


In [63]:
%%jedit -h -o hour_tasks_demo.html -w hour_tasks.html
// tämä esimerkki on asynkronisesta kutsusta, mutta kokeillaan ensin synkronista
<button onclick="show_tasks()">get tasks</button>
<p id="demo"></p>

<script>
function get_tasks() {
var url = "http://localhost:5000/get_tasks";
var xhr = new XMLHttpRequest();
xhr.open("GET", url, true);
xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");
xhr.onreadystatechange = function () {
   if (xhr.readyState === 4) {
      console.log(xhr.status);
//       console.log(xhr.responseText);
//       document.getElementById("demo").innerHTML = 'status=' + xhr.status + 'response=' + xhr.responseText;
       tasks_response = JSON.parse(xhr.responseText);
       tasks = tasks_response['tasks'];
//       document.getElementById("demo").innerHTML = JSON.stringify(tasks[0]);
       return tasks
   }};

xhr.send(null);
}

function show_tasks() {
    tasks = get_tasks();
    document.getElementById("demo").innerHTML = JSON.stringify(tasks[0]);
}
</script>


<button onclick="show_tasks()">get tasks</button>
<p id="demo"></p>

<script>
function get_tasks() {
var url = "http://localhost:5000/get_tasks";
var xhr = new XMLHttpRequest();
xhr.open("GET", url, true);
xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");
xhr.onreadystatechange = function () {
   if (xhr.readyState === 4) {
      console.log(xhr.status);
//       console.log(xhr.responseText);
//       document.getElementById("demo").innerHTML = 'status=' + xhr.status + 'response=' + xhr.responseText;
       tasks_response = JSON.parse(xhr.responseText);
       tasks = tasks_response['tasks'];
//       document.getElementById("demo").innerHTML = JSON.stringify(tasks[0]);
       return tasks
   }};

xhr.send(null);
}

function show_tasks() {
    tasks = get_tasks();
    document.getElementById("demo").innerHTML = JSON.stringify(tasks[0]);
}
</script>
Opening page file:////Users/rook/Documents/github/python/hour_reporting/hour_t

127.0.0.1 - - [18/Oct/2021 20:03:26] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:26] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:27] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:27] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:27] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:27] "GET /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:28] "OPTIONS /get_tasks HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 20:03:28] "GET /get_tasks HTTP/1.1" 200 -


In [29]:
%%sql my_conn -d ./sql -w get_logs -ww
select
    *
from
    time_log

Writing SQL query in file : ./sql/get_logs.sql


,id,jira_id,description,log_time
0,1,test_ticket,test,2021-10-19 19:27:15.707040
1,2,test_ticket,test,2021-10-19 19:27:19.574387
2,3,ABC-2,,2021-10-19 19:27:31.302530
3,4,ABC-2,,2021-10-19 19:27:59.612246
4,5,ABC-2,,2021-10-19 19:28:54.432035
...,...,...,...,...
64,66,ABC-2,,2021-10-21 19:00:43.534840
65,67,UNIT-A,,2021-10-21 19:00:57.591279
66,68,UUSI-2,No vähän kaikkea,2021-10-21 19:01:12.697145
67,69,UUSI-1,aa,2021-10-21 21:50:14.980370


In [54]:
%%js
function insert_log() {
var url = "http://localhost:5000/insert_log";

var xhr = new XMLHttpRequest();
xhr.open("POST", url);

xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");

xhr.onreadystatechange = function () {
   if (xhr.readyState === 4) {
      console.log(xhr.status);
      console.log(xhr.responseText);
   }};

var data = "{}";

xhr.send(data);
}

<IPython.core.display.Javascript object>

In [77]:
%%jedit -h -o temp.html -w hour_interface.html
<!DOCTYPE html>
<html>
<body>

<p>Hour interface</p>

<button onclick="myFunction()">insert hour</button>

<p id="demo"></p>

<script>
function insert_log() {
var url = "http://localhost:5000/insert_log";

var xhr = new XMLHttpRequest();
xhr.open("POST", url);

xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");

xhr.onreadystatechange = function () {
   if (xhr.readyState === 4) {
      console.log(xhr.status);
      console.log(xhr.responseText);
   }};

var data = "{}";

xhr.send(data);
}

function myFunction() {
    insert_log();
  document.getElementById("demo").innerHTML = "Hello World";
}
</script>


<!DOCTYPE html>
<html>
<body>

<p>Hour interface</p>

<button onclick="myFunction()">insert hour</button>

<p id="demo"></p>

<script>
function insert_log() {
var url = "http://localhost:5000/insert_log";

var xhr = new XMLHttpRequest();
xhr.open("POST", url);

xhr.setRequestHeader("Accept", "application/json");
xhr.setRequestHeader("Content-Type", "application/json");

xhr.onreadystatechange = function () {
   if (xhr.readyState === 4) {
      console.log(xhr.status);
      console.log(xhr.responseText);
   }};

var data = "{}";

xhr.send(data);
}

function myFunction() {
    insert_log();
  document.getElementById("demo").innerHTML = "Hello World";
}
</script>
Opening page file:////Users/rook/Documents/github/python/hour_reporting/temp.html in the browser
